In [40]:
import csv

with open('test.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')

    latitude_list = ""
    longitude_list = ""
    counter = 0
    for row in csv_reader:
        cleared = row[8].replace("],[", " -1 ").replace("[[", "").replace("]]", "")
        if cleared != "POLYLINE":
            splitted = cleared.split(" -1 ")
            for entry in splitted:
                data = entry.split(",")
                latitude_list += data[1] + ","
                longitude_list += data[0] + ","
            print("latitude_list" + str(counter) + ": " + latitude_list)
            print("longitude_list" + str(counter) + ": " + longitude_list)
            latitude_list = ""
            longitude_list = ""
            counter += 1


latitude_list0: 41.148522,41.148639,41.148855,41.148927,41.148963,41.148954,41.14872,41.147847,41.14746,41.147154,41.146623,
longitude_list0: -8.585676,-8.585712,-8.585685,-8.58573,-8.585982,-8.586396,-8.586072,-8.586324,-8.586999,-8.586576,-8.584884,
latitude_list1: 41.14557,41.145579,41.145768,41.146191,41.146758,41.147118,41.147127,41.147532,41.148117,41.148351,41.148576,41.14926,41.149899,41.149899,41.149962,41.150979,41.151915,41.152788,41.153319,41.154345,41.154372,41.155353,41.156775,41.158197,41.159943,41.160555,41.1606,41.160645,41.160807,41.161176,41.161248,41.161293,41.161266,41.161239,41.161194,41.161275,41.162049,41.162832,41.163111,41.163597,
longitude_list1: -8.610876,-8.610858,-8.610903,-8.610444,-8.609445,-8.608896,-8.608968,-8.608707,-8.608347,-8.608149,-8.608041,-8.607654,-8.607348,-8.607393,-8.607357,-8.606817,-8.606358,-8.605719,-8.604981,-8.604783,-8.604828,-8.604801,-8.604648,-8.604522,-8.604513,-8.604378,-8.604378,-8.604369,-8.60436,-8.604162,-8.604126,-8.60409,

In [1]:
import gmplot 

gmap3 = gmplot.GoogleMapPlotter( 41.148522, -8.585676, 15) 
  
# trip0
latitude_list0 = [ 41.148522, 41.148639, 41.148855, 41.148927, 41.148963 , 41.148954, 41.14872, 41.147847, 41.14746, 41.147154, 41.146623] 
longitude_list0 = [ -8.585676, -8.585712, -8.585685, -8.58573 , -8.585982, -8.586396, -8.586072, -8.586324, -8.586999, -8.586576, -8.584884 ] 
gmap3.scatter( latitude_list0, longitude_list0, '# FF0000', size = 2.5, marker = False ) 
gmap3.plot(latitude_list0, longitude_list0, 'cornflowerblue', edge_width = 2.5) 

# trip1
latitude_list1 = [41.14557,41.145579,41.145768,41.146191,41.146758,41.147118,41.147127,41.147532,41.148117,41.148351,41.148576,41.14926,41.149899,41.149899,41.149962,41.150979,41.151915,41.152788,41.153319,41.154345,41.154372,41.155353,41.156775,41.158197,41.159943,41.160555,41.1606,41.160645,41.160807,41.161176,41.161248,41.161293,41.161266,41.161239,41.161194,41.161275,41.162049,41.162832,41.163111,41.163597] 
longitude_list1 = [ -8.610876,-8.610858,-8.610903,-8.610444,-8.609445,-8.608896,-8.608968,-8.608707,-8.608347,-8.608149,-8.608041,-8.607654,-8.607348,-8.607393,-8.607357,-8.606817,-8.606358,-8.605719,-8.604981,-8.604783,-8.604828,-8.604801,-8.604648,-8.604522,-8.604513,-8.604378,-8.604378,-8.604369,-8.60436,-8.604162,-8.604126,-8.60409,-8.60409,-8.604108,-8.604126,-8.604135,-8.60391,-8.602929,-8.602551,-8.601894] 
gmap3.scatter( latitude_list1, longitude_list1, '# FF0000', size = 2.5, marker = False ) 
gmap3.plot(latitude_list1, longitude_list1, 'cornflowerblue', edge_width = 2.5) 

# trip5
latitude_list5 = [41.178249,41.178249,41.178231,41.178141,41.178114,41.177889,41.177817,41.17797,41.178042,41.177394,41.176152,41.176125,41.176107,41.175684,41.174127,41.172588,41.171922,41.171895,41.171904,41.171913,41.171913,41.171913,41.171922,41.171922,41.171931,41.171931,41.171931,41.171922,41.171922,41.171913,41.171913,41.171922,41.171913,41.171913,41.171913,41.171913,41.171913,41.171904,41.171904,41.171913,41.171904,41.171895,41.171895,41.171886,41.171886,41.171886,41.171877,41.171877,41.171895,41.171895,41.171895,41.171913,41.171913,41.171922,41.171922,41.171931,41.171922,41.171922,41.171931,41.171922,41.171922,41.171922,41.171931,41.171922,41.171922,41.171913,41.171913,41.171913,41.171913,41.171913,41.171913,41.171904,41.171904,41.171904,41.171913,41.171913,41.171922,41.171922,41.171922,41.171913,41.171904,41.171904,41.171895,41.171895,41.171895,41.171895,41.171895,41.171904,41.171913,41.171913,41.171913,41.171913,41.171904,41.171904,41.171913,41.171922,41.171922,41.171931,41.171931,41.17194,41.171949,41.171949,41.171949,41.17194,41.171931,41.171931,41.171931,41.171931,41.171895,41.171895,41.171868,41.171859,41.171886,41.171859,41.171886,41.171904,41.171904,41.171904,41.171904,41.171904,41.171859,41.171832,41.171886,41.171904,41.171904,41.171904,41.171913,41.171922,41.171922,41.171913,41.171913,41.171913,41.171922,41.17194,41.171958,41.171967,41.171958]
longitude_list5 = [-8.630613,-8.630613,-8.630739,-8.631513,-8.631306,-8.630028,-8.629425,-8.629173,-8.629056,-8.62866,-8.627958,-8.627967,-8.627985,-8.627778,-8.627157,-8.626275,-8.626221,-8.626419,-8.626419,-8.626428,-8.626428,-8.626419,-8.62641,-8.626401,-8.62641,-8.62641,-8.626419,-8.626419,-8.626419,-8.626419,-8.626437,-8.626446,-8.626446,-8.626446,-8.626437,-8.626428,-8.626437,-8.626437,-8.626437,-8.626428,-8.626428,-8.626437,-8.626446,-8.626446,-8.626446,-8.626437,-8.626437,-8.626446,-8.626446,-8.626446,-8.626446,-8.626446,-8.626446,-8.626437,-8.626446,-8.626455,-8.626464,-8.626464,-8.626464,-8.626464,-8.626455,-8.626455,-8.626446,-8.626446,-8.626437,-8.626437,-8.626437,-8.626446,-8.626455,-8.626464,-8.626464,-8.626464,-8.626455,-8.626455,-8.626446,-8.626446,-8.626437,-8.626437,-8.626446,-8.626446,-8.626446,-8.626455,-8.626464,-8.626464,-8.626464,-8.626473,-8.626473,-8.626464,-8.626464,-8.626455,-8.626455,-8.626455,-8.626455,-8.626446,-8.626437,-8.626437,-8.626437,-8.626446,-8.626446,-8.626446,-8.626446,-8.626455,-8.626455,-8.626464,-8.626473,-8.626473,-8.626473,-8.626509,-8.626689,-8.626698,-8.62668,-8.626662,-8.626878,-8.626842,-8.626833,-8.626842,-8.626842,-8.626851,-8.626842,-8.626833,-8.626824,-8.626779,-8.626509,-8.626455,-8.626455,-8.626455,-8.626428,-8.62641,-8.62641,-8.626419,-8.626419,-8.626428,-8.626419,-8.626401,-8.626392,-8.626401,-8.62641] 
gmap3.scatter( latitude_list5, longitude_list5, '# FF0000', size = 2.5, marker = False ) 
gmap3.plot(latitude_list5, longitude_list5, 'cornflowerblue', edge_width = 2.5) 
  
gmap3.draw( "map.html" )

In [ ]:
[[-8.585676,41.148522],[-8.585712,41.148639],[-8.585685,41.148855],[-8.58573,41.148927],[-8.585982,41.148963],[-8.586396,41.148954],[-8.586072,41.14872],[-8.586324,41.147847],[-8.586999,41.14746],[-8.586576,41.147154],[-8.584884,41.146623]]